In [2]:
## Check Pre-Requisites

In [3]:
%store -r setup_instance_check_passed

In [4]:
try:
    setup_instance_check_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Instance Check.")
    print("+++++++++++++++++++++++++++++++")

In [5]:
print(setup_instance_check_passed)

True


In [6]:
%store -r setup_dependencies_passed

In [7]:
try:
    setup_dependencies_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++")

In [8]:
print(setup_dependencies_passed)

True


In [9]:
%store -r setup_s3_bucket_passed

In [10]:
try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++")

In [11]:
print(setup_s3_bucket_passed)

True


In [12]:
%store -r setup_iam_roles_passed

In [13]:
try:
    setup_iam_roles_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup IAM Roles.")
    print("+++++++++++++++++++++++++++++++")

In [14]:
print(setup_iam_roles_passed)

True


In [15]:
if not setup_instance_check_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Instance Check.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
if not setup_dependencies_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
if not setup_s3_bucket_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
if not setup_iam_roles_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup IAM Roles.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [16]:
## Install Packages 

In [17]:
# pip install boto3 --upgrade


In [18]:
# Import Packages

In [2]:
import boto3
import sagemaker
import pandas as pd
import os
import random
from PIL import Image
import io
import numpy as np
import shutil

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
# Definitions 
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
## Copy Public Bucket to Private Bucket 

In [19]:
# Set S3 Source Location (ads508project)

In [20]:
public= "s3://project508data"

In [21]:
%store public

Stored 'public' (str)


In [22]:
# Set S3 Destination Location (ads508-xrays)

In [23]:
private = "s3://508-xray-project".format(bucket)
print(private)

s3://508-xray-project


In [24]:
%store private

Stored 'private' (str)


In [25]:
# copy from public to private 

In [26]:
# !aws s3 cp --recursive $public/ $private/

In [ ]:
## Copy Covid folder after relabeling from Jessica's bucket to mine 

In [ ]:
# Copying new covid labeled data into covid folder 
# setting S3 source location
public_path = "s3://project508data/covid/" 

# setting up S3 destination location
private_path = "s3://508-xray-project/covid".format(bucket) 

# copy data from one s3 bucket into another S3 bucket
# !aws s3 cp  $public_path $private_path --recursive

In [22]:
# Creating a copy of covid folder in case of errors 

# Initialize S3 client
s3 = boto3.client('s3')

def copy_folder(source_bucket, source_folder, destination_bucket, destination_folder):
    # List objects in the source folder
    response = s3.list_objects_v2(Bucket=source_bucket, Prefix=source_folder)

    # Copy each object to the destination folder
    for obj in response.get('Contents', []):
        # Get object key
        object_key = obj['Key']

        # Construct destination object key
        destination_key = object_key.replace(source_folder, destination_folder, 1)

        # Copy object to destination folder
        s3.copy_object(
            CopySource={'Bucket': source_bucket, 'Key': object_key},
            Bucket=destination_bucket,
            Key=destination_key
        )

# Set source and destination folder names
# source_bucket_name = '508-xray-project' 
# source_folder_prefix = 'covid/'
# destination_bucket_name = '508-xray-project'
# destination_folder_prefix = 'covid_copy/'

# Copy folder contents
copy_folder(source_bucket_name, source_folder_prefix, destination_bucket_name, destination_folder_prefix)

print("Sampling folder copied successfully.")

Sampling folder copied successfully.


In [ ]:
# Copying new pneumonia labeled data into pnuemonia folder from Jessica's bucket 
# setting S3 source location
public_path = "s3://project508data/pneumonia/" 

# setting up S3 destination location
private_path = "s3://508-xray-project/pneumonia".format(bucket) 

# copy data from one s3 bucket into another S3 bucket
# !aws s3 cp  $public_path $private_path --recursive

In [23]:
# Creating a copy of pneumonia folder in case of errors 

# Initialize S3 client
s3 = boto3.client('s3')

def copy_folder(source_bucket, source_folder, destination_bucket, destination_folder):
    # List objects in the source folder
    response = s3.list_objects_v2(Bucket=source_bucket, Prefix=source_folder)

    # Copy each object to the destination folder
    for obj in response.get('Contents', []):
        # Get object key
        object_key = obj['Key']

        # Construct destination object key
        destination_key = object_key.replace(source_folder, destination_folder, 1)

        # Copy object to destination folder
        s3.copy_object(
            CopySource={'Bucket': source_bucket, 'Key': object_key},
            Bucket=destination_bucket,
            Key=destination_key
        )

# Set source and destination folder names
# source_bucket_name = '508-xray-project'
# source_folder_prefix = 'pneumonia/'
# destination_bucket_name = '508-xray-project'
# destination_folder_prefix = 'pneumonia_copy/'

# Copy folder contents
copy_folder(source_bucket_name, source_folder_prefix, destination_bucket_name, destination_folder_prefix)

print("Sampling folder copied successfully.")

Sampling folder copied successfully.


In [28]:
# Copying new NORMAL labeled data into normal folder from Jessica's bucket 

# setting S3 source location
public_path = "s3://project508data/normal/" 

# setting up S3 destination location
private_path = "s3://508-xray-project/normal".format(bucket) 

# copy data from one s3 bucket into another S3 bucket
# !aws s3 cp  $public_path $private_path --recursive

copy: s3://project508data/normal/normal_10.jpeg to s3://508-xray-project/normal/normal_10.jpeg
copy: s3://project508data/normal/normal_1.jpeg to s3://508-xray-project/normal/normal_1.jpeg
copy: s3://project508data/normal/normal_100.jpeg to s3://508-xray-project/normal/normal_100.jpeg
copy: s3://project508data/normal/normal_1006.jpeg to s3://508-xray-project/normal/normal_1006.jpeg
copy: s3://project508data/normal/normal_1004.jpeg to s3://508-xray-project/normal/normal_1004.jpeg
copy: s3://project508data/normal/normal_1001.jpeg to s3://508-xray-project/normal/normal_1001.jpeg
copy: s3://project508data/normal/normal_1000.jpeg to s3://508-xray-project/normal/normal_1000.jpeg
copy: s3://project508data/normal/normal_1009.jpeg to s3://508-xray-project/normal/normal_1009.jpeg
copy: s3://project508data/normal/normal_1008.jpeg to s3://508-xray-project/normal/normal_1008.jpeg
copy: s3://project508data/normal/normal_1007.jpeg to s3://508-xray-project/normal/normal_1007.jpeg
copy: s3://project508d

In [29]:
# Creating a copy of NORMAL folder in case of errors 

# Initialize S3 client
s3 = boto3.client('s3')

def copy_folder(source_bucket, source_folder, destination_bucket, destination_folder):
    # List objects in the source folder
    response = s3.list_objects_v2(Bucket=source_bucket, Prefix=source_folder)

    # Copy each object to the destination folder
    for obj in response.get('Contents', []):
        # Get object key
        object_key = obj['Key']

        # Construct destination object key
        destination_key = object_key.replace(source_folder, destination_folder, 1)

        # Copy object to destination folder
        s3.copy_object(
            CopySource={'Bucket': source_bucket, 'Key': object_key},
            Bucket=destination_bucket,
            Key=destination_key
        )

# Set source and destination folder names
# source_bucket_name = '508-xray-project'
# source_folder_prefix = 'normal/'
# destination_bucket_name = '508-xray-project'
# destination_folder_prefix = 'normal_copy/'

# Copy folder contents
copy_folder(source_bucket_name, source_folder_prefix, destination_bucket_name, destination_folder_prefix)

print("Sampling folder copied successfully.")

Sampling folder copied successfully.


In [30]:
# List Folders in 508-xray-project bucket
# Create an S3 client
s3 = boto3.client('s3')

bucket_name = '508-xray-project'

# List folders (or prefixes) in the bucket
response = s3.list_objects_v2(Bucket=bucket_name, Delimiter='/')

# Print the folder names (prefixes)
if 'CommonPrefixes' in response:
    for prefix in response['CommonPrefixes']:
        print(prefix['Prefix'])
else:
    print("No folders found in the bucket.")

covid/
covid_copy/
normal/
normal_copy/
pneumonia/
pneumonia_copy/


In [32]:
# List all files in the NORMAL folder 
bucket_name = '508-xray-project'

# Specify the prefix (folder) within the bucket
folder_prefix = 'normal/'

# List objects in the specified folder of the bucket
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_prefix)

# Print the object keys
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No objects found in the folder.")

normal/normal_1.jpeg
normal/normal_10.jpeg
normal/normal_100.jpeg
normal/normal_1000.jpeg
normal/normal_1001.jpeg
normal/normal_1002.jpeg
normal/normal_1003.jpeg
normal/normal_1004.jpeg
normal/normal_1005.jpeg
normal/normal_1006.jpeg
normal/normal_1007.jpeg
normal/normal_1008.jpeg
normal/normal_1009.jpeg
normal/normal_101.jpeg
normal/normal_1010.jpeg
normal/normal_1011.jpeg
normal/normal_1012.jpeg
normal/normal_1013.jpeg
normal/normal_1014.jpeg
normal/normal_1015.jpeg
normal/normal_1016.jpeg
normal/normal_1017.jpeg
normal/normal_1018.jpeg
normal/normal_1019.jpeg
normal/normal_102.jpeg
normal/normal_1020.jpeg
normal/normal_1021.jpeg
normal/normal_1022.jpeg
normal/normal_1023.jpeg
normal/normal_1024.jpeg
normal/normal_1025.jpeg
normal/normal_1026.jpeg
normal/normal_1027.jpeg
normal/normal_1028.jpeg
normal/normal_1029.jpeg
normal/normal_103.jpeg
normal/normal_1030.jpeg
normal/normal_1031.jpeg
normal/normal_1032.jpeg
normal/normal_1033.jpeg
normal/normal_1034.jpeg
normal/normal_1035.jpeg
n

In [33]:
# Count Number of Files in Normal Folder after copying from Jessica's bucket 
bucket_name = '508-xray-project'

# Specify the prefix (folder) within the bucket
folder_prefix = 'normal/'

# Initialize the counter for objects
num_files = 0

# Pagination loop to list all objects in the specified folder
paginator = s3.get_paginator('list_objects_v2')
for page in paginator.paginate(Bucket=bucket_name, Prefix=folder_prefix):
    if 'Contents' in page:
        num_files += len(page['Contents'])

print("Number of files in the folder:", num_files)

Number of files in the folder: 1340


In [36]:
# Count number of Covid files in folder after Copying from Jessica's bucket - including .txt files 
# Specify bucket 
bucket_name = '508-xray-project'

# Specify the prefix (folder) within the bucket
folder_prefix = 'covid/'

# Create an S3 client
s3 = boto3.client('s3')

# Initialize the counter for objects
num_files = 0

# Pagination loop to list all objects in the specified folder
paginator = s3.get_paginator('list_objects_v2')
for page in paginator.paginate(Bucket=bucket_name, Prefix=folder_prefix):
    if 'Contents' in page:
        num_files += len(page['Contents'])

print("Number of files in the folder:", num_files)

Number of files in the folder: 3631


In [37]:
# Count number of relabeled Covid xrays in folder after Copying from Jessica's bucket - including .txt files 
bucket_name = '508-xray-project'

# Specify the prefix (folder) within the bucket
folder_prefix = 'covid/train/'

# Initialize the counter for objects
num_files = 0

# Pagination loop to list all objects in the specified folder
paginator = s3.get_paginator('list_objects_v2')
for page in paginator.paginate(Bucket=bucket_name, Prefix=folder_prefix):
    if 'Contents' in page:
        num_files += len(page['Contents'])

print("Number of files in the folder:", num_files)

Number of files in the folder: 3628


In [39]:
# Count number of relabeled pneumonia xrays in folder after Copying from Jessica's bucket 
bucket_name = '508-xray-project'

# Specify the prefix (folder) within the bucket
folder_prefix = 'pneumonia/'

# Initialize the counter for objects
num_files = 0

# Pagination loop to list all objects in the specified folder
paginator = s3.get_paginator('list_objects_v2')
for page in paginator.paginate(Bucket=bucket_name, Prefix=folder_prefix):
    if 'Contents' in page:
        num_files += len(page['Contents'])

print("Number of files in the folder:", num_files)

Number of files in the folder: 8546


In [35]:
# Create random sample to achieve a subset 3000 files, 1000 from each folder  
# Initialize S3 client
s3 = boto3.client('s3')

def random_sampling(source_bucket, source_folder, destination_bucket, destination_folder, num_samples):
    # List objects in the source folder
    response = s3.list_objects_v2(Bucket=source_bucket, Prefix=source_folder)

    # Extract object keys
    object_keys = [obj['Key'] for obj in response.get('Contents', [])]

    # Calculate the proportion of samples to take from this folder
    proportion = min(num_samples, len(object_keys)) / len(object_keys)

    # Randomly select a subset of object keys
    sampled_object_keys = random.sample(object_keys, min(num_samples, len(object_keys)))

    # Copy sampled objects to the destination folder
    for key in sampled_object_keys:
        source_key = key
        destination_key = key.replace(source_folder, destination_folder, 1)
        s3.copy_object(CopySource={'Bucket': source_bucket, 'Key': source_key}, Bucket=destination_bucket, Key=destination_key)

# Set parameters
source_bucket = '508-xray-project'
destination_bucket = '508-xray-project'
num_samples_per_folder = 1000

# Sampling for each folder
folders_to_sample = ['normal/', 'pneumonia/', 'covid/train/']
for folder in folders_to_sample:
    random_sampling(source_bucket, folder, destination_bucket, 'sampling_folder/', num_samples_per_folder)


In [5]:
# Count number of files in sampling_folder to confirm the sampling plan 
s3 = boto3.client('s3')
bucket_name = '508-xray-project'

# Specify the prefix (folder) within the bucket
folder_prefix = 'sampling_folder/'

# Initialize the counter for objects
num_files = 0

# Pagination loop to list all objects in the specified folder
paginator = s3.get_paginator('list_objects_v2')
for page in paginator.paginate(Bucket=bucket_name, Prefix=folder_prefix):
    if 'Contents' in page:
        num_files += len(page['Contents'])

print("Number of files in the folder:", num_files)

Number of files in the folder: 3000


In [17]:

def count_files_in_folder(bucket_name, folder_prefix):
    # Create a Boto3 client for S3
    s3_client = boto3.client('s3')
    
    # Initialize counters
    normal_count = 0
    pneumonia_count = 0
    covid_count = 0
   
 # List objects in the specified folder
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=folder_prefix)
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                # Extract the object key
                obj_key = obj['Key']
                # Count files based on their category
                if 'normal' in obj_key:
                    normal_count += 1
                elif 'pneumonia' in obj_key:
                    pneumonia_count += 1
                elif 'covid' in obj_key:
                    covid_count += 1
    return normal_count, pneumonia_count, covid_count

# Specify your S3 bucket name and folder prefix
bucket_name = '508-xray-project'
folder_prefix = 'sampling_folder/'

# Call the function to count files
normal_count, pneumonia_count, covid_count = count_files_in_folder(bucket_name, folder_prefix)

# Print the counts
print("Normal files:", normal_count)
print("Pneumonia files:", pneumonia_count)
print("COVID files:", covid_count)

Normal files: 1000
Pneumonia files: 1000
COVID files: 1000


In [44]:
# Creating a copy of sampling_folder in case of errors 

# Initialize S3 client
s3 = boto3.client('s3')

def copy_folder(source_bucket, source_folder, destination_bucket, destination_folder):
    # List objects in the source folder
    response = s3.list_objects_v2(Bucket=source_bucket, Prefix=source_folder)

    # Copy each object to the destination folder
    for obj in response.get('Contents', []):
        # Get object key
        object_key = obj['Key']

        # Construct destination object key
        destination_key = object_key.replace(source_folder, destination_folder, 1)

        # Copy object to destination folder
        s3.copy_object(
            CopySource={'Bucket': source_bucket, 'Key': object_key},
            Bucket=destination_bucket,
            Key=destination_key
        )

# Set source and destination folder names
# source_bucket_name = '508-xray-project'
# source_folder_prefix = 'sampling_folder/'
# destination_bucket_name = '508-xray-project'
# destination_folder_prefix = 'sampling_folder_copy/'

# Copy folder contents
copy_folder(source_bucket_name, source_folder_prefix, destination_bucket_name, destination_folder_prefix)

print("Sampling folder copied successfully.")

Sampling folder copied successfully.


In [46]:
# Normalize the pixels of each image on a sale of [0, 1]
# Initialize S3 client
s3 = boto3.client('s3')

def normalize_image(bucket_name, folder_prefix):
    # List objects in the specified folder of the bucket
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_prefix)

    # Process each object in the folder
    for obj in response.get('Contents', []):
        # Get object key
        object_key = obj['Key']

        # Download image from S3
        response = s3.get_object(Bucket=bucket_name, Key=object_key)
        image_bytes = response['Body'].read()

        # Normalize image
        image = Image.open(io.BytesIO(image_bytes))
        image_array = np.array(image)
        normalized_image_array = image_array / 255.0  

        # Convert normalized image array back to image
        normalized_image = Image.fromarray((normalized_image_array * 255).astype(np.uint8))

        # Save normalized image to bytes
        with io.BytesIO() as output:
            normalized_image.save(output, format='JPEG')
            normalized_image_bytes = output.getvalue()

        # Upload normalized image back to S3
        s3.put_object(Bucket=bucket_name, Key=object_key, Body=normalized_image_bytes)

# Set bucket name and folder prefix
bucket_name = '508-xray-project'
folder_prefix = 'sampling_folder/'

# Normalize images in the specified folder
normalize_image(bucket_name, folder_prefix)

In [48]:
from io import BytesIO

In [49]:
# Function to resize images in the sampling folder
def resize_images_in_folder(bucket_name, folder_prefix, new_size):
    # List objects in the specified folder
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_prefix)
    
    # Iterate over objects in the folder
    for obj in response.get('Contents', []):
        # Get the object key
        key = obj['Key']
        
        # Download image from S3
        response = s3.get_object(Bucket=bucket_name, Key=key)
        image_bytes = response['Body'].read()
        
        # Resize image
        image = Image.open(BytesIO(image_bytes))
        resized_image = image.resize(new_size)
        
        # Save resized image to bytes buffer
        with BytesIO() as output:
            resized_image.save(output, format=image.format)
            resized_image_bytes = output.getvalue()
        
        # Upload resized image back to S3
        s3.put_object(Bucket=bucket_name, Key=key, Body=resized_image_bytes)

# Define the bucket name and folder prefix
bucket_name = '508-xray-project'
folder_prefix = 'sampling_folder/'

# Define the new size for the resized images
new_size = (224, 224)  

# Resize images in the sampling folder
resize_images_in_folder(bucket_name, folder_prefix, new_size)

print("Images resized successfully.")

Images resized successfully.


In [50]:
import shutil

In [54]:
# Initialize S3 client
s3 = boto3.client('s3')

# Define the bucket name and folder prefixes
bucket_name = '508-xray-project'
source_folder_prefix = 'sampling_folder/'
train_folder_prefix = 'train/'
test_folder_prefix = 'test/'
validation_folder_prefix = 'validation/'

# Function to copy files from source folder to destination folder
def copy_files(source_bucket, source_prefix, destination_bucket, destination_prefix, files):
    for file in files:
        source_key = source_prefix + file
        destination_key = destination_prefix + file
        s3.copy_object(
            CopySource={'Bucket': source_bucket, 'Key': source_key},
            Bucket=destination_bucket,
            Key=destination_key
        )

# List files in the source folder
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=source_folder_prefix)
files = [obj['Key'].split('/')[-1] for obj in response.get('Contents', [])]

# Calculate the number of files for each set
num_files = len(files)
num_train = int(0.9 * num_files)
num_test = int(0.05 * num_files)
num_validation = num_files - num_train - num_test

# Split files into train, test, and validation sets
train_files = files[:num_train]
test_files = files[num_train:num_train + num_test]
validation_files = files[num_train + num_test:]

# Copy files to train, test, and validation folders
copy_files(bucket_name, source_folder_prefix, bucket_name, train_folder_prefix, train_files)
copy_files(bucket_name, source_folder_prefix, bucket_name, test_folder_prefix, test_files)
copy_files(bucket_name, source_folder_prefix, bucket_name, validation_folder_prefix, validation_files)

print("Data split into train, test, and validation sets successfully.")

Data split into train, test, and validation sets successfully.


In [22]:
# Connect to S3
s3 = boto3.client('s3')

# Define bucket name and folder paths
bucket_name = '508-xray-project'
input_folder_path = 'sampling_folder'
output_train_folder_path = 'train_folder'
output_validation_folder_path = 'validation_folder'
output_test_folder_path = 'test_folder'

# List files in the input folder
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=input_folder_path)
files = [obj['Key'] for obj in response.get('Contents', [])]

# Shuffle the files
random.shuffle(files)

# Calculate the number of files for each category
num_files = len(files)
num_normal_files = num_pneumonia_files = num_covid_files = 0
for file in files:
    if 'normal' in file:
        num_normal_files += 1
    elif 'pneumonia' in file:
        num_pneumonia_files += 1
    elif 'covid' in file:
        num_covid_files += 1
        
# Calculate the number of files for each set
num_train_files = int(0.9 * num_files)
num_validation_files = num_test_files = int(0.05 * num_files)

# Create output folders
s3.put_object(Bucket=bucket_name, Key=output_train_folder_path + '/')
s3.put_object(Bucket=bucket_name, Key=output_validation_folder_path + '/')
s3.put_object(Bucket=bucket_name, Key=output_test_folder_path + '/')

# Copy files to output folders
def copy_files(files, output_folder_path, num_files_to_copy):
    for file in files[:num_files_to_copy]:
        output_key = os.path.join(output_folder_path, os.path.basename(file))
        s3.copy_object(CopySource={'Bucket': bucket_name, 'Key': file}, Bucket=bucket_name, Key=output_key)
copy_files(files, output_train_folder_path, num_train_files)
copy_files(files[num_train_files:], output_validation_folder_path, num_validation_files)
copy_files(files[num_train_files + num_validation_files:], output_test_folder_path, num_test_files)
print("Files copied successfully.")

Files copied successfully.


In [4]:

def count_files_in_folder(bucket_name, folder_prefix):
    # Create a Boto3 client for S3
    s3_client = boto3.client('s3')
    
    # Initialize counters
    normal_count = 0
    pneumonia_count = 0
    covid_count = 0
   
 # List objects in the specified folder
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=folder_prefix)
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                # Extract the object key
                obj_key = obj['Key']
                # Count files based on their category
                if 'normal' in obj_key:
                    normal_count += 1
                elif 'pneumonia' in obj_key:
                    pneumonia_count += 1
                elif 'covid' in obj_key:
                    covid_count += 1
    return normal_count, pneumonia_count, covid_count

# Specify your S3 bucket name and folder prefix
bucket_name = '508-xray-project'
folder_prefix = 'train_folder/'

# Call the function to count files
normal_count, pneumonia_count, covid_count = count_files_in_folder(bucket_name, folder_prefix)

# Print the counts
print("Normal files:", normal_count)
print("Pneumonia files:", pneumonia_count)
print("COVID files:", covid_count)

Normal files: 0
Pneumonia files: 0
COVID files: 900


In [55]:
# Define S3 bucket and folder path
bucket_name = 'project508data'
folder_path = 'test/'

# List objects in the specified S3 folder
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_path)
# Define renaming conventions and counters
covid_counter = 1
pneumonia_counter = 1
normal_counter = 1

# Iterate through objects in the bucket
for obj in response['Contents']:
    key = obj['Key']
    # Check if the object's key contains the word "covid" and rename accordingly
    if 'covid' in key:
        # extracting the file name and extension
        filename, extension = key.rsplit('.', 1)
        # Rename object
        new_filename = f'x_{covid_counter}.{extension}'
        new_key = folder_path + new_filename
        s3.copy_object(Bucket=bucket_name, CopySource=f'{bucket_name}/{key}', Key=new_key)
        s3.delete_object(Bucket=bucket_name, Key=key)
        covid_counter += 1
        
    # Check if the object's key contains the word "pneumonia" and rename accordingly
    elif 'pneumonia' in key:
        # extracting the file name and extension
        filename, extension = key.rsplit('.', 1)
        # Rename object
        new_filename = f'y_{pneumonia_counter}.{extension}'
        new_key = folder_path + new_filename
        s3.copy_object(Bucket=bucket_name, CopySource=f'{bucket_name}/{key}', Key=new_key)
        s3.delete_object(Bucket=bucket_name,Key=key)
        pneumonia_counter += 1
        
    # Check if the object's key contains the word "normal" and rename accordingly
    elif 'normal' in key:
        # extracting the file name and extension
        filename, extension = key.rsplit('.', 1)
        # Rename object
        new_filename = f'z_{normal_counter}.{extension}'
        new_key = folder_path + new_filename
        s3.copy_object(Bucket=bucket_name, CopySource=f'{bucket_name}/{key}', Key=new_key)
        s3.delete_object(Bucket=bucket_name, Key=key)
        normal_counter += 1
    else:
        print("Done Renaming") 

Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming
Done Renaming


In [56]:
# List all files in the NORMAL folder 
bucket_name = '508-xray-project'

# Specify the prefix (folder) within the bucket
folder_prefix = 'test/'

# List objects in the specified folder of the bucket
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=folder_prefix)

# Print the object keys
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No objects found in the folder.")

test/covid_1809.jpg
test/covid_181.jpg
test/covid_1810.jpg
test/covid_1811.jpg
test/covid_1812.jpg
test/covid_1813.jpg
test/covid_1814.jpg
test/covid_1815.jpg
test/covid_1816.jpg
test/covid_1817.jpg
test/covid_1818.jpg
test/covid_1819.jpg
test/covid_182.jpg
test/covid_1820.jpg
test/covid_1821.jpg
test/covid_1822.jpg
test/covid_1823.jpg
test/covid_1824.jpg
test/covid_1825.jpg
test/covid_1826.jpg
test/covid_1827.jpg
test/covid_1828.jpg
test/covid_1829.jpg
test/covid_183.jpg
test/covid_1830.jpg
test/covid_1831.jpg
test/covid_1832.jpg
test/covid_1833.jpg
test/covid_1834.jpg
test/covid_1835.jpg
test/covid_1836.jpg
test/covid_1837.jpg
test/covid_1838.jpg
test/covid_1839.jpg
test/covid_184.jpg
test/covid_1840.jpg
test/covid_1841.jpg
test/covid_1842.jpg
test/covid_1843.jpg
test/covid_1844.jpg
test/covid_1845.jpg
test/covid_1846.jpg
test/covid_1847.jpg
test/covid_1848.jpg
test/covid_1849.jpg
test/covid_185.jpg
test/covid_1850.jpg
test/covid_1851.jpg
test/covid_1852.jpg
test/covid_1853.jpg
